# HDFM Framework Complete Tutorial

**Hierarchical Dendritic Forest Management: A Computational Framework for Optimal Corridor Design**

This tutorial covers all major features of the HDFM framework:

1. **Getting Started** - Installation and basic concepts
2. **Landscape Creation** - Synthetic and custom landscapes
3. **Network Construction** - Dendritic (MST) and alternative topologies
4. **Entropy Calculations** - Information-theoretic optimization
5. **Species-Specific Planning** - Guild parameters from Hart (2024)
6. **Width Optimization** - Corridor widths under landscape budget constraints
7. **Genetic Viability** - Effective population size (Nₑ) calculations
8. **Robustness Analysis** - Network resilience with strategic loops
9. **Climate Adaptation** - Backwards temporal optimization
10. **Full Workflow** - Complete corridor design example

---

**Version:** 0.2.1  
**Author:** Justin Hart  
**Framework:** https://github.com/jdhart81/hdfm-framework

## 1. Getting Started

First, let's import the HDFM framework and verify the installation.

In [ ]:
# Core imports
import numpy as np
import matplotlib.pyplot as plt

# HDFM imports
import hdfm
from hdfm import (
    # Landscape
    Landscape, SyntheticLandscape, Patch,
    
    # Network construction
    build_dendritic_network, compare_network_topologies,
    
    # Entropy
    calculate_entropy, calculate_entropy_rate,
    
    # Species
    SPECIES_GUILDS, print_guild_summary, get_guild,
    
    # Optimization
    DendriticOptimizer, BackwardsOptimizer, WidthOptimizer,
    ClimateScenario, check_allocation_constraint,
    
    # Genetics
    calculate_effective_population_size, check_genetic_viability,
    
    # Robustness
    calculate_robustness_metrics, add_strategic_loops, pareto_frontier_analysis,
    
    # Visualization
    plot_landscape, plot_network, plot_optimization_trace
)

print(f"HDFM Framework v{hdfm.__version__}")
print(f"Author: {hdfm.__author__}")
print("\n✓ All imports successful!")

## 2. Landscape Creation

Landscapes are collections of forest patches that need to be connected. HDFM supports both synthetic (random) landscapes for testing and custom landscapes from real data.

### 2.1 Synthetic Landscapes

Create random landscapes for algorithm testing and validation.

In [ ]:
# Create a synthetic landscape with 15 forest patches
landscape = SyntheticLandscape(
    n_patches=15,           # Number of forest patches
    extent=10000.0,         # Landscape size in meters (10km x 10km)
    random_seed=42          # For reproducibility
)

print(f"Created landscape with {len(landscape.patches)} patches")
print(f"Landscape extent: {landscape.extent}m x {landscape.extent}m")
print(f"\nPatch summary:")
for i, patch in enumerate(landscape.patches[:5]):
    print(f"  Patch {patch.id}: ({patch.x:.0f}, {patch.y:.0f}), area={patch.area:.0f}ha, quality={patch.quality:.2f}")
print(f"  ... and {len(landscape.patches) - 5} more patches")

In [ ]:
# Visualize the landscape
fig, ax = plt.subplots(figsize=(10, 10))
plot_landscape(landscape, ax=ax)
ax.set_title('Synthetic Landscape (15 patches)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### 2.2 Custom Landscapes

Create landscapes from your own GIS data or field surveys.

In [ ]:
# Define patches manually (e.g., from GIS data)
custom_patches = [
    Patch(id=0, x=1000, y=2000, area=50, quality=0.9),   # High-quality core habitat
    Patch(id=1, x=3500, y=3000, area=75, quality=0.85),  # Large secondary habitat
    Patch(id=2, x=2000, y=5000, area=30, quality=0.7),   # Smaller stepping stone
    Patch(id=3, x=5000, y=4500, area=100, quality=0.95), # Largest high-quality patch
    Patch(id=4, x=4000, y=1500, area=40, quality=0.6),   # Lower quality edge habitat
    Patch(id=5, x=6000, y=3000, area=55, quality=0.8),   # Eastern corridor node
]

custom_landscape = Landscape(custom_patches)

print(f"Custom landscape with {len(custom_landscape.patches)} patches")
print(f"Total habitat area: {sum(p.area for p in custom_landscape.patches):.0f} ha")
print(f"Mean quality: {np.mean([p.quality for p in custom_landscape.patches]):.2f}")

## 3. Network Construction

The core theorem of HDFM: **Dendritic (tree-like) networks minimize landscape entropy.**

Dendritic networks are constructed using the Minimum Spanning Tree (MST) algorithm.

In [ ]:
# Build the optimal dendritic network
network = build_dendritic_network(landscape)

print(f"Dendritic network constructed:")
print(f"  Nodes (patches): {len(landscape.patches)}")
print(f"  Edges (corridors): {len(network.edges)}")
print(f"  Total corridor length: {network.total_length:.0f}m")
print(f"\nCorridors:")
for i, j in list(network.edges)[:5]:
    dist = landscape.distance(i, j)
    print(f"  Patch {i} ↔ Patch {j}: {dist:.0f}m")

In [ ]:
# Visualize the dendritic network
fig, ax = plt.subplots(figsize=(10, 10))
plot_network(landscape, network, ax=ax, title='Optimal Dendritic Corridor Network')
plt.tight_layout()
plt.show()

### 3.1 Compare Network Topologies

Demonstrate that dendritic networks outperform alternative topologies.

In [ ]:
# Compare 5 different network topologies
results = compare_network_topologies(landscape)

print("Network Topology Comparison")
print("=" * 60)
print(f"{'Topology':<20} {'Entropy':<12} {'Corridors':<12} {'vs Dendritic':<15}")
print("-" * 60)

dendritic_H = results['dendritic']['H_total']
for topo, data in sorted(results.items(), key=lambda x: x[1]['H_total']):
    H = data['H_total']
    n_edges = data['n_edges']
    diff = ((H - dendritic_H) / dendritic_H) * 100 if topo != 'dendritic' else 0
    marker = "★ OPTIMAL" if topo == 'dendritic' else f"+{diff:.1f}%"
    print(f"{topo.capitalize():<20} {H:<12.3f} {n_edges:<12} {marker:<15}")

## 4. Entropy Calculations

The HDFM framework uses information-theoretic entropy to quantify landscape connectivity:

$$H(L) = H_{mov} + \lambda_1 C(L) + \lambda_2 F(L) + \lambda_3 D(L)$$

Where:
- $H_{mov}$ = Movement entropy (Shannon entropy of dispersal probabilities)
- $C(L)$ = Connectivity constraint penalty
- $F(L)$ = Forest topology penalty (favors dendritic structures)
- $D(L)$ = Disturbance response penalty (path length)

In [ ]:
# Calculate entropy for the dendritic network
H_total, components = calculate_entropy(landscape, network.edges)

print("Entropy Breakdown")
print("=" * 40)
print(f"Movement entropy (H_mov):     {components['H_mov']:.4f}")
print(f"Connectivity penalty (C):     {components['C']:.4f}")
print(f"Topology penalty (F):         {components['F']:.4f}")
print(f"Response time penalty (D):    {components['D']:.4f}")
print("-" * 40)
print(f"Total entropy H(L):           {H_total:.4f}")

### 4.1 Width-Dependent Entropy

Corridor width affects movement success. Wider corridors have higher success rates:

$$\phi(w) = 1 - \exp(-\gamma(w - w_{min}))$$

In [ ]:
# Calculate entropy with corridor widths
corridor_widths = {edge: 200.0 for edge in network.edges}  # 200m wide corridors
guild = SPECIES_GUILDS['medium_mammals']

H_with_widths, components = calculate_entropy(
    landscape=landscape,
    edges=network.edges,
    corridor_widths=corridor_widths,
    species_guild=guild
)

print(f"Entropy with 200m corridors: {H_with_widths:.4f}")
print(f"Movement success at 200m: {guild.movement_success(200):.1%}")

## 5. Species-Specific Planning

Different species have different corridor requirements. HDFM includes 4 guilds from Table 2 of Hart (2024).

In [ ]:
# Display all available species guilds
print_guild_summary()

In [ ]:
# Compare corridor width requirements across species
print("\nCorridor Width Requirements by Species Guild")
print("=" * 60)
print(f"{'Guild':<20} {'Critical Width':<15} {'For 90% Success':<15}")
print("-" * 60)

for name, guild in SPECIES_GUILDS.items():
    critical = guild.w_crit
    width_90 = guild.required_width_for_success(0.90)
    print(f"{name:<20} {critical:<15.0f}m {width_90:<15.0f}m")

In [ ]:
# Plot movement success curves for all guilds
widths = np.linspace(50, 600, 100)

fig, ax = plt.subplots(figsize=(10, 6))

colors = ['green', 'blue', 'orange', 'red']
for (name, guild), color in zip(SPECIES_GUILDS.items(), colors):
    success = [guild.movement_success(w) for w in widths]
    ax.plot(widths, success, label=name.replace('_', ' ').title(), 
            color=color, linewidth=2)
    # Mark critical width
    ax.axvline(x=guild.w_crit, color=color, linestyle='--', alpha=0.5)

ax.set_xlabel('Corridor Width (m)', fontsize=12)
ax.set_ylabel('Movement Success Probability', fontsize=12)
ax.set_title('Movement Success by Corridor Width and Species Guild', fontsize=14, fontweight='bold')
ax.legend(loc='lower right')
ax.grid(True, alpha=0.3)
ax.set_ylim(0, 1.05)
plt.tight_layout()
plt.show()

## 6. Width Optimization

Optimize corridor widths under landscape allocation constraints (typically 20-30% of landscape area).

In [ ]:
# Optimize widths for medium mammals with 25% landscape budget
guild = SPECIES_GUILDS['medium_mammals']

optimizer = WidthOptimizer(
    landscape=landscape,
    edges=network.edges,
    species_guild=guild,
    beta=0.25  # 25% of landscape can be corridors
)

result = optimizer.optimize()

print("Width Optimization Results")
print("=" * 50)
print(f"Species guild: {guild.name}")
print(f"Landscape budget: 25%")
print(f"Optimization entropy: {result.entropy:.4f}")
print(f"\nOptimized corridor widths:")
for edge, width in list(result.corridor_widths.items())[:5]:
    print(f"  {edge}: {width:.1f}m")
if len(result.corridor_widths) > 5:
    print(f"  ... and {len(result.corridor_widths) - 5} more")

In [ ]:
# Verify allocation constraint is satisfied
satisfied, corridor_area, total_area = check_allocation_constraint(
    landscape=landscape,
    edges=network.edges,
    corridor_widths=result.corridor_widths,
    beta=0.25
)

allocation_pct = (corridor_area / total_area) * 100
print(f"\nAllocation Constraint Check")
print(f"  Corridor area: {corridor_area/10000:.2f} ha")
print(f"  Landscape area: {total_area/10000:.2f} ha")
print(f"  Allocation: {allocation_pct:.1f}% (budget: 25%)")
print(f"  Constraint satisfied: {'✓ Yes' if satisfied else '✗ No'}")

## 7. Genetic Viability Assessment

Calculate effective population size (Nₑ) to assess long-term genetic viability using the island model:

$$N_e(A,w) = \frac{[\sum_i n_i]^2}{\sum_i n_i^2 + \sum_i \sum_{j \neq i} 2n_i n_j (1 - F_{ij}(A,w))}$$

In [ ]:
# Calculate effective population size
guild = SPECIES_GUILDS['large_carnivores']
corridor_widths = {edge: 300.0 for edge in network.edges}

Ne, components = calculate_effective_population_size(
    landscape=landscape,
    edges=network.edges,
    corridor_widths=corridor_widths,
    species_guild=guild
)

print("Genetic Viability Assessment")
print("=" * 50)
print(f"Species guild: {guild.name}")
print(f"Corridor widths: 300m")
print(f"\nEffective population size (Nₑ): {Ne:.1f}")
print(f"Total census population (N): {components['total_N']:.0f}")
print(f"Nₑ/N ratio: {Ne/components['total_N']:.2f}")

In [ ]:
# Check genetic viability against 50/500 rule
viable, threshold, message = check_genetic_viability(Ne, guild)

print(f"\nViability Check (50/500 Rule)")
print(f"  Guild threshold: {threshold:.0f}")
print(f"  Status: {message}")

In [ ]:
# Compare Nₑ across different corridor widths
widths_to_test = [100, 150, 200, 250, 300, 350, 400]
Ne_values = []

for w in widths_to_test:
    corridor_widths = {edge: float(w) for edge in network.edges}
    Ne, _ = calculate_effective_population_size(
        landscape, network.edges, corridor_widths, species_guild=guild
    )
    Ne_values.append(Ne)

# Plot
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(widths_to_test, Ne_values, 'bo-', linewidth=2, markersize=8)
ax.axhline(y=guild.Ne_threshold, color='red', linestyle='--', label=f'Viability threshold ({guild.Ne_threshold})')
ax.axhline(y=50, color='orange', linestyle=':', label='Minimum viable (50)')
ax.set_xlabel('Corridor Width (m)', fontsize=12)
ax.set_ylabel('Effective Population Size (Nₑ)', fontsize=12)
ax.set_title(f'Genetic Viability vs Corridor Width ({guild.name})', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 8. Robustness Analysis

Dendritic networks (trees) are efficient but vulnerable - removing any edge disconnects the network. Strategic loops can improve robustness.

In [ ]:
# Calculate robustness metrics for the dendritic network
metrics = calculate_robustness_metrics(landscape, network.edges)

print("Robustness Analysis: Dendritic Network")
print("=" * 50)
print(f"Two-edge connectivity (ρ₂): {metrics.two_edge_connectivity:.3f}")
print(f"Failure probability (P_fail): {metrics.failure_probability:.3f}")
print(f"Number of bridges: {metrics.n_bridges}")
print(f"Mean redundancy score: {metrics.mean_redundancy:.3f}")
print(f"\n⚠️  All edges are bridges in a tree - network is vulnerable!")

In [ ]:
# Add strategic loops to improve robustness
robust_edges = add_strategic_loops(
    landscape=landscape,
    edges=network.edges,
    n_loops=3,  # Add 3 strategic loops
    criterion='betweenness'  # Prioritize high-betweenness edges
)

# Re-calculate robustness
robust_metrics = calculate_robustness_metrics(landscape, robust_edges)

print("\nRobustness After Adding 3 Strategic Loops")
print("=" * 50)
print(f"Corridors: {len(network.edges)} → {len(robust_edges)}")
print(f"Two-edge connectivity (ρ₂): {metrics.two_edge_connectivity:.3f} → {robust_metrics.two_edge_connectivity:.3f}")
print(f"Failure probability (P_fail): {metrics.failure_probability:.3f} → {robust_metrics.failure_probability:.3f}")
print(f"Bridges: {metrics.n_bridges} → {robust_metrics.n_bridges}")

In [ ]:
# Visualize original vs robust network
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

plot_network(landscape, network, ax=axes[0], 
             title=f'Dendritic (MST)\nρ₂={metrics.two_edge_connectivity:.2f}, P_fail={metrics.failure_probability:.2f}')
plot_network(landscape, robust_edges, ax=axes[1],
             title=f'With Strategic Loops (+3)\nρ₂={robust_metrics.two_edge_connectivity:.2f}, P_fail={robust_metrics.failure_probability:.2f}')

fig.suptitle('Robustness Enhancement via Strategic Loops', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Explore entropy-robustness tradeoff (Pareto frontier)
pareto_results = pareto_frontier_analysis(landscape, max_loops=8)

print("\nPareto Frontier: Entropy vs Robustness")
print("=" * 60)
print(f"{'Loops':<8} {'Edges':<8} {'Entropy':<12} {'ρ₂':<10} {'P_fail':<10}")
print("-" * 60)

for r in pareto_results:
    print(f"{r['n_loops']:<8} {r['n_edges']:<8} {r['entropy']:<12.3f} {r['rho2']:<10.3f} {r['p_fail']:<10.3f}")

## 9. Climate Adaptation: Backwards Optimization

Design corridors that remain effective under future climate scenarios by optimizing **backwards from 2100 to present**.

In [ ]:
# Define a climate scenario (RCP 8.5-like trajectory)
scenario = ClimateScenario(
    years=[2025, 2050, 2075, 2100],
    temperature_changes=[0.0, 1.5, 2.5, 3.5],  # °C warming
    precipitation_changes=[0.0, -5.0, -10.0, -15.0]  # % change
)

print("Climate Scenario (RCP 8.5-like)")
print("=" * 40)
for year, temp, precip in zip(scenario.years, scenario.temperature_changes, scenario.precipitation_changes):
    print(f"  {year}: +{temp:.1f}°C, {precip:+.1f}% precipitation")

In [ ]:
# Run backwards optimization with width scheduling
guild = SPECIES_GUILDS['medium_mammals']

backwards_optimizer = BackwardsOptimizer(
    landscape=landscape,
    scenario=scenario,
    species_guild=guild,
    beta=0.25,  # 25% landscape allocation
    optimize_widths=True
)

result = backwards_optimizer.optimize(max_iterations=50)

print("\nBackwards Optimization Results")
print("=" * 50)
print(f"Final network entropy: {result.entropy:.4f}")
print(f"Corridors established: {len(result.network.edges)}")
print(f"Optimization iterations: {result.iterations}")

In [ ]:
# View the temporal corridor schedule
if result.corridor_schedule:
    print("\nTemporal Corridor Schedule")
    print("=" * 50)
    for year, edges in zip(scenario.years, result.corridor_schedule):
        print(f"  {year}: {len(edges)} corridors")

# View width schedule if available
if result.width_schedule:
    print("\nWidth Schedule by Year")
    print("=" * 50)
    for year, widths in zip(scenario.years, result.width_schedule):
        mean_width = np.mean(list(widths.values()))
        print(f"  {year}: mean width = {mean_width:.1f}m")

## 10. Complete Workflow Example

Putting it all together: A complete corridor design workflow for conservation planning.

In [ ]:
print("="*70)
print("COMPLETE HDFM CORRIDOR DESIGN WORKFLOW")
print("="*70)

# Step 1: Create/load landscape
print("\n[1] Creating landscape...")
landscape = SyntheticLandscape(n_patches=20, extent=15000, random_seed=123)
print(f"    ✓ {len(landscape.patches)} patches, {landscape.extent/1000:.0f}km extent")

# Step 2: Build dendritic network
print("\n[2] Building optimal dendritic network...")
network = build_dendritic_network(landscape)
print(f"    ✓ {len(network.edges)} corridors, {network.total_length/1000:.1f}km total")

# Step 3: Select target species
print("\n[3] Selecting target species guild...")
guild = SPECIES_GUILDS['medium_mammals']
print(f"    ✓ {guild.name}: critical width {guild.w_crit}m, Nₑ threshold {guild.Ne_threshold}")

# Step 4: Optimize corridor widths
print("\n[4] Optimizing corridor widths (25% landscape budget)...")
width_opt = WidthOptimizer(landscape, network.edges, guild, beta=0.25)
width_result = width_opt.optimize()
mean_width = np.mean(list(width_result.corridor_widths.values()))
print(f"    ✓ Mean width: {mean_width:.1f}m, entropy: {width_result.entropy:.4f}")

# Step 5: Assess genetic viability
print("\n[5] Assessing genetic viability...")
Ne, _ = calculate_effective_population_size(
    landscape, network.edges, width_result.corridor_widths, guild
)
viable, _, msg = check_genetic_viability(Ne, guild)
print(f"    ✓ Nₑ = {Ne:.1f}: {msg}")

# Step 6: Assess robustness
print("\n[6] Assessing network robustness...")
metrics = calculate_robustness_metrics(landscape, network.edges)
print(f"    ✓ ρ₂ = {metrics.two_edge_connectivity:.3f}, P_fail = {metrics.failure_probability:.3f}")

# Step 7: Add loops if needed
if metrics.failure_probability > 0.3:
    print("\n[7] Adding strategic loops for robustness...")
    robust_edges = add_strategic_loops(landscape, network.edges, n_loops=3)
    robust_metrics = calculate_robustness_metrics(landscape, robust_edges)
    print(f"    ✓ P_fail reduced: {metrics.failure_probability:.3f} → {robust_metrics.failure_probability:.3f}")
else:
    print("\n[7] Robustness acceptable, no loops needed")
    robust_edges = network.edges

# Step 8: Climate adaptation
print("\n[8] Running backwards climate optimization...")
scenario = ClimateScenario(
    years=[2025, 2050, 2075, 2100],
    temperature_changes=[0.0, 1.5, 2.5, 3.5],
    precipitation_changes=[0.0, -5.0, -10.0, -15.0]
)
back_opt = BackwardsOptimizer(landscape, scenario, guild, beta=0.25, optimize_widths=True)
climate_result = back_opt.optimize(max_iterations=30)
print(f"    ✓ Climate-adapted network ready, entropy: {climate_result.entropy:.4f}")

print("\n" + "="*70)
print("WORKFLOW COMPLETE")
print("="*70)

In [ ]:
# Final visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 14))

# Original landscape
plot_landscape(landscape, ax=axes[0, 0])
axes[0, 0].set_title('1. Landscape (20 patches)', fontsize=12, fontweight='bold')

# Dendritic network
plot_network(landscape, network, ax=axes[0, 1], 
             title='2. Dendritic Network (MST)')

# Robust network with loops
plot_network(landscape, robust_edges, ax=axes[1, 0],
             title='3. Robust Network (+3 loops)')

# Climate-adapted network
plot_network(landscape, climate_result.network, ax=axes[1, 1],
             title='4. Climate-Adapted Network')

fig.suptitle('HDFM Corridor Design Workflow', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## Summary

This tutorial demonstrated the complete HDFM framework:

1. **Landscapes** - Create synthetic or custom patch networks
2. **Networks** - Build optimal dendritic (MST) corridor networks
3. **Entropy** - Calculate information-theoretic landscape entropy
4. **Species** - Use guild-specific parameters for corridor design
5. **Width Optimization** - Allocate corridor widths under budget constraints
6. **Genetics** - Assess long-term population viability (Nₑ)
7. **Robustness** - Add strategic loops for network resilience
8. **Climate Adaptation** - Design for future climate scenarios

### Key Findings

- **Dendritic networks minimize entropy** - Tree structures are optimal for corridor networks
- **Species-specific design matters** - Different guilds need different corridor widths
- **Robustness-entropy tradeoff** - Loops improve resilience but increase entropy
- **Climate adaptation is critical** - Backwards optimization ensures long-term viability

### Next Steps

- Apply to your own landscape data
- Explore multi-species planning
- Integrate with GIS workflows
- Run Monte Carlo validation

---

**Reference:** Hart, J. (2024). "Hierarchical Dendritic Forest Management: A Vision for Technology-Enabled Landscape Conservation." *EcoEvoRxiv*.